# Jigsaw generation #2

In [1]:
from collections import Counter

import numpy as np

from tqdm import tqdm

from jigsaw import *
from jigsaw.default import *
from jigsaw.svg import *

In [2]:
def is_unique(horizontal_edges, vertical_edges, *, constraints="border"):
    H, _ = horizontal_edges.shape
    _, W = vertical_edges.shape
    pieces = grid_to_pieces(horizontal_edges, vertical_edges, opposite)
    return has_unique_solution(H, W, pieces, opposite, flip, constraints=constraints)


def search(sampler, *validators, max_iterations=None):

    # Loop until a grid with a unique solution is found
    i = 0
    with tqdm(total=max_iterations) as progress:
        while max_iterations is None or i < max_iterations:

            # Sample grid, according to rules
            horizontal_edges, vertical_edges = sampler()
            
            # Ask validators for compliance
            try:
                for validator in validators:
                    if not validator(horizontal_edges, vertical_edges):
                        break
                else:
                    return horizontal_edges, vertical_edges
            finally:
                progress.update(1)
            
            i += 1

In [3]:
def count_solutions(horizontal_edges, vertical_edges, *, constraints="border"):
    H, _ = horizontal_edges.shape
    _, W = vertical_edges.shape
    pieces = grid_to_pieces(horizontal_edges, vertical_edges, opposite)
    iterator = iterate_unique_solutions(H, W, pieces, opposite, flip, constraints=constraints)
    count = sum(1 for _ in iterator)
    return count

def display_solutions(horizontal_edges, vertical_edges, *, constraints="border"):
    H, _ = horizontal_edges.shape
    _, W = vertical_edges.shape
    pieces = grid_to_pieces(horizontal_edges, vertical_edges, opposite)
    for h, v in iterate_unique_solutions(H, W, pieces, opposite, flip, constraints=constraints):
        display_grid(h, v)

In [4]:
def has_unique_pieces(horizontal_edges, vertical_edges):
    pieces = grid_to_pieces(horizontal_edges, vertical_edges, opposite)
    pieces = canonize_piece(pieces, flip)
    pieces = pieces.reshape(-1, 4)
    unique_pieces = np.unique(pieces, axis=0)
    return pieces.shape[0] == unique_pieces.shape[0]

## 1. 4x4 with 8 corners (no constraint)

In [5]:
# Choose edge types
edge_types = np.array([
    CENTERED_SMALL_MALE,
    LEFT_SMALL_MALE,
])

# Automatically expand types to match combinations
edge_types = expand_edge_types(edge_types, opposite, flip)

# Show actual types
edge_names[edge_types]

array(['CENTERED_SMALL_MALE', 'CENTERED_SMALL_FEMALE', 'RIGHT_SMALL_MALE',
       'RIGHT_SMALL_FEMALE', 'LEFT_SMALL_MALE', 'LEFT_SMALL_FEMALE'],
      dtype='<U22')

In [6]:
# Generate grid
H, W = 4, 4
generator = np.random.default_rng(42)
def sampler():
    horizontal_edges, vertical_edges = sample_random_grid(H, W, edge_types, generator=generator)
    horizontal_edges[1:-1, 2] = FLAT
    vertical_edges[2, 1:-1] = FLAT
    return horizontal_edges, vertical_edges
horizontal_edges, vertical_edges = search(sampler, is_unique)

30it [00:04,  6.04it/s]


In [7]:
# Show it
display_grid(horizontal_edges, vertical_edges)

In [8]:
# Keep it for later
pieces_4x4_cross = grid_to_pieces(horizontal_edges, vertical_edges, opposite)

## 2. 5x5 with nested corners

In [9]:
# This will be a 5x5 grid, with 4 additional corned inside
H, W = 5, 5
horizontal_edges, vertical_edges = sample_random_grid(H, W, np.array([FLAT_ALTERNATIVE]))
horizontal_edges[[1, 1, 3, 3], [1, 4, 1, 4]] = FLAT
vertical_edges[[1, 1, 4, 4], [1, 3, 1, 3]] = FLAT

# First, make sure that the overall layout can only be solved one way
assert is_unique(horizontal_edges, vertical_edges)

# Show it, to illustrate the purpose
display_grid(horizontal_edges, vertical_edges)

In [10]:
# Outer perimeter cannot contain any centered small pin
# This makes it easier to identify which corners are inside
outer_edge_types = np.array([
    LEFT_SMALL_MALE,
    LEFT_SMALL_FEMALE,
    RIGHT_SMALL_MALE,
    RIGHT_SMALL_FEMALE,
] * 2 + [
    DOUBLE_SMALL_MALE,
    DOUBLE_SMALL_FEMALE,
    TWISTED_SMALL_MALE,
    TWISTED_SMALL_FEMALE,
])

generator = np.random.default_rng(13746)

def sampler():
    
    # Start with empty (i.e. flat-only) 5x5 grid
    H = W = 5
    horizontal_edges = np.zeros((H, W + 1), dtype=np.uint8)
    vertical_edges = np.zeros((H + 1, W), dtype=np.uint8)
    
    # Sample outer perimeter
    horizontal_edges[[0, 0, 0, 0, 4, 4, 4, 4], [1, 2, 3, 4, 1, 2, 3, 4]] = generator.choice(outer_edge_types, size=8)
    vertical_edges[[1, 2, 3, 4, 1, 2, 3, 4], [0, 0, 0, 0, 4, 4, 4, 4]] = generator.choice(outer_edge_types, size=8)
    
    # Inner area is set to wildcard, for the moment
    horizontal_edges[[1, 1, 2, 2, 2, 2, 3, 3], [2, 3, 1, 2, 3, 4, 2, 3]] = FLAT_ALTERNATIVE
    vertical_edges[[2, 3, 1, 2, 3, 4, 2, 3], [1, 1, 2, 2, 2, 2, 3, 3]] = FLAT_ALTERNATIVE
    
    return horizontal_edges, vertical_edges

outer_horizontal_edges, outer_vertical_edges = search(sampler, is_unique)

234it [00:06, 33.76it/s]


In [11]:
# Show it
display_grid(outer_horizontal_edges, outer_vertical_edges)

In [12]:
inner_edge_types = np.array([
    CENTERED_SMALL_MALE,
    CENTERED_SMALL_FEMALE,
    LEFT_SMALL_MALE,
    LEFT_SMALL_FEMALE,
    RIGHT_SMALL_MALE,
    RIGHT_SMALL_FEMALE,
] * 4 + [
    DOUBLE_SMALL_MALE,
    DOUBLE_SMALL_FEMALE,
    #TWISTED_SMALL_MALE,
    #TWISTED_SMALL_FEMALE,
])

generator = np.random.default_rng(123)

def sampler():
    
    # Start with selected solution
    horizontal_edges = outer_horizontal_edges.copy()
    vertical_edges = outer_vertical_edges.copy()
    
    # Inner can contain twisted pins as well
    horizontal_edges[[1, 1, 2, 2, 2, 2, 3, 3], [2, 3, 1, 2, 3, 4, 2, 3]] = generator.choice(inner_edge_types, size=8)
    vertical_edges[[2, 3, 1, 2, 3, 4, 2, 3], [1, 1, 2, 2, 2, 2, 3, 3]] = generator.choice(inner_edge_types, size=8)
    
    # Place centered ones manually, to ensure diversity
    horizontal_edges[1, 2] = CENTERED_SMALL_MALE
    horizontal_edges[3, 2] = CENTERED_SMALL_MALE
    horizontal_edges[3, 3] = CENTERED_SMALL_MALE
    vertical_edges[2, 3] = CENTERED_SMALL_FEMALE
    vertical_edges[3, 1] = CENTERED_SMALL_FEMALE
    vertical_edges[1, 2] = CENTERED_SMALL_FEMALE
    
    return horizontal_edges, vertical_edges

horizontal_edges, vertical_edges = search(sampler, is_unique)

8it [00:08,  1.11s/it]


In [13]:
# Show it
display_grid(horizontal_edges, vertical_edges)

In [14]:
# Focus on the inner part
inner_horizontal_edges = horizontal_edges[1:-1, 1:-1]
inner_vertical_edges = vertical_edges[1:-1, 1:-1]

# Show it
display_grid(inner_horizontal_edges, inner_vertical_edges)

In [15]:
# Checking unicity on this subset is a bit more tricky
# We need to allow any pins on the central edges
horizontal_constraints = np.full((3, 3 + 1), -1)
vertical_constraints = np.full((3 + 1, 3), -1)
horizontal_constraints[0, [0, -1]] = FLAT
horizontal_constraints[2, [0, -1]] = FLAT
vertical_constraints[[0, -1], 0] = FLAT
vertical_constraints[[0, -1], 2] = FLAT
constraints = horizontal_constraints, vertical_constraints

# Hopefully there is only a unique solution
count_solutions(inner_horizontal_edges, inner_vertical_edges, constraints=constraints)

1

In [16]:
# Keep it for later
pieces_5x5_nested = grid_to_pieces(horizontal_edges, vertical_edges, opposite)

## 3. 4x4 with pure square

In [17]:
# This will be a 4x4 grid
H, W = 4, 4
horizontal_edges, vertical_edges = sample_random_grid(H, W, np.array([FLAT_ALTERNATIVE]))
horizontal_edges[0, 1] = FLAT
vertical_edges[1, 0] = FLAT

# First, make sure that the overall layout can only be solved one way only
assert is_unique(horizontal_edges, vertical_edges)

# Show it, to illustrate the purpose
display_grid(horizontal_edges, vertical_edges)

In [18]:
# Choose edge types
edge_types = np.array([
    CENTERED_SMALL_MALE,
    #LEFT_SMALL_MALE,
    CENTERED_MEDIUM_MALE,
])
edge_types = expand_edge_types(edge_types, opposite, flip)

# Generate grid
generator = np.random.default_rng(123123)
def sampler():
    horizontal_edges, vertical_edges = sample_random_grid(H, W, edge_types, generator=generator)
    
    # Add square in upper-left corner
    horizontal_edges[0, 1] = FLAT
    vertical_edges[1, 0] = FLAT
    
    # Central pieces have male pins "radiating" outside
    horizontal_edges[1:3, 1] = generator.choice([CENTERED_SMALL_MALE, CENTERED_MEDIUM_MALE], size=2)
    horizontal_edges[1:3, 3] = generator.choice([CENTERED_SMALL_FEMALE, CENTERED_MEDIUM_FEMALE], size=2)
    vertical_edges[1, 1:3] = generator.choice([CENTERED_SMALL_FEMALE, CENTERED_MEDIUM_FEMALE], size=2)
    vertical_edges[3, 1:3] = generator.choice([CENTERED_SMALL_MALE, CENTERED_MEDIUM_MALE], size=2)

    # Corners that are not next to the square are male only (i.e. they cannot connect with the radiating center)
    horizontal_edges[[0, 3], 3] = generator.choice([CENTERED_SMALL_MALE, CENTERED_MEDIUM_MALE], size=2)
    horizontal_edges[3, 1] = generator.choice([CENTERED_SMALL_FEMALE, CENTERED_MEDIUM_FEMALE])
    vertical_edges[3, [0, 3]] = generator.choice([CENTERED_SMALL_FEMALE, CENTERED_MEDIUM_FEMALE], size=2)
    vertical_edges[1, 3] = generator.choice([CENTERED_SMALL_MALE, CENTERED_MEDIUM_MALE])
    
    return horizontal_edges, vertical_edges
horizontal_edges, vertical_edges = search(sampler, has_unique_pieces, is_unique)

4850it [00:07, 630.55it/s]


In [19]:
# Show it
display_grid(horizontal_edges, vertical_edges)

In [20]:
# Keep it for later
pieces_4x4_pure = grid_to_pieces(horizontal_edges, vertical_edges, opposite)

## 4. 4x4 with more edges

In [21]:
# This will be a 4x4 grid
H, W = 4, 4
horizontal_edges, vertical_edges = sample_random_grid(H, W, np.array([FLAT_ALTERNATIVE]))
vertical_edges[2, 1:-1] = FLAT

# First, make sure that the overall layout can only be solved one way
assert is_unique(horizontal_edges, vertical_edges)

# Show it, to illustrate the purpose
display_grid(horizontal_edges, vertical_edges)

In [22]:
# Choose edge types
edge_types = np.array([
    CENTERED_SMALL_MALE,
    LEFT_SMALL_MALE,
])

# Automatically expand types to match combinations
edge_types = expand_edge_types(edge_types, opposite, flip)

# Show actual types
edge_names[edge_types]

array(['CENTERED_SMALL_MALE', 'CENTERED_SMALL_FEMALE', 'RIGHT_SMALL_MALE',
       'RIGHT_SMALL_FEMALE', 'LEFT_SMALL_MALE', 'LEFT_SMALL_FEMALE'],
      dtype='<U22')

In [23]:
# Generate grid
H, W = 4, 4
generator = np.random.default_rng(78125)
def sampler():
    horizontal_edges, vertical_edges = sample_random_grid(H, W, edge_types, generator=generator)
    
    # Add an horizontal split, so that the inner pieces are also edge pieces
    vertical_edges[2, 1:-1] = FLAT
    
    # Force the two "internal" edge pairs to have double pins
    horizontal_edges[1:-1, 2] = DOUBLE_SMALL_MALE
    
    # Force one end of these "pairs" to have female centered pins
    horizontal_edges[1:-1, 1] = CENTERED_SMALL_FEMALE
    
    # And then force corners to not provide any male centered pins
    # Therefore, these "pairs" cannot be connected to a corner
    horizontal_edges[[0, 3], 1] = generator.choice([CENTERED_SMALL_MALE, LEFT_SMALL_MALE, LEFT_SMALL_FEMALE, RIGHT_SMALL_MALE, RIGHT_SMALL_FEMALE], size=2)
    horizontal_edges[[0, 3], 3] = generator.choice([CENTERED_SMALL_FEMALE, LEFT_SMALL_MALE, LEFT_SMALL_FEMALE, RIGHT_SMALL_MALE, RIGHT_SMALL_FEMALE], size=2)
    vertical_edges[1, [0, 3]] = generator.choice([CENTERED_SMALL_FEMALE, LEFT_SMALL_MALE, LEFT_SMALL_FEMALE, RIGHT_SMALL_MALE, RIGHT_SMALL_FEMALE], size=2)
    vertical_edges[3, [0, 3]] = generator.choice([CENTERED_SMALL_MALE, LEFT_SMALL_MALE, LEFT_SMALL_FEMALE, RIGHT_SMALL_MALE, RIGHT_SMALL_FEMALE], size=2)
    
    return horizontal_edges, vertical_edges
horizontal_edges, vertical_edges = search(sampler, has_unique_pieces, is_unique)

6it [00:00, 200.44it/s]


In [24]:
# Show it
display_grid(horizontal_edges, vertical_edges)

In [25]:
# Keep it for later
pieces_4x4_edge = grid_to_pieces(horizontal_edges, vertical_edges, opposite)

## 5. 4x4 with 6 corners

In [26]:
# This will be a 4x4 grid
H, W = 4, 4
horizontal_edges, vertical_edges = sample_random_grid(H, W, np.array([FLAT_ALTERNATIVE]))
horizontal_edges[0, 2] = FLAT

# First, make sure that the overall layout can only be solved one way
assert is_unique(horizontal_edges, vertical_edges)

# Show it, to illustrate the purpose
display_grid(horizontal_edges, vertical_edges)

In [27]:
# Choose edge types
edge_types = np.array([
    CENTERED_SMALL_MALE,
    LEFT_SMALL_MALE,
])

# Automatically expand types to match combinations
edge_types = expand_edge_types(edge_types, opposite, flip)

# Show actual types
edge_names[edge_types]

array(['CENTERED_SMALL_MALE', 'CENTERED_SMALL_FEMALE', 'RIGHT_SMALL_MALE',
       'RIGHT_SMALL_FEMALE', 'LEFT_SMALL_MALE', 'LEFT_SMALL_FEMALE'],
      dtype='<U22')

In [28]:
# Generate grid
H, W = 4, 4
generator = np.random.default_rng(78125)
def sampler():
    horizontal_edges, vertical_edges = sample_random_grid(H, W, edge_types, generator=generator)
    
    # Add a single flat edge, so that the top edge pieces become corners
    horizontal_edges[0, 2] = FLAT
    
    # Add some double pins, in such a way that there cannot be ambiguity
    # Pairs can be easily identified (but not necessarily orientation)
    horizontal_edges[3, 1] = DOUBLE_SMALL_MALE
    vertical_edges[1, 1] = DOUBLE_SMALL_MALE
    vertical_edges[3, 2] = DOUBLE_SMALL_MALE
    
    return horizontal_edges, vertical_edges
horizontal_edges, vertical_edges = search(sampler, has_unique_pieces, is_unique)

3it [00:00, 142.85it/s]


In [29]:
# Show it
display_grid(horizontal_edges, vertical_edges)

In [30]:
# Keep it for later
pieces_4x4_6_corners = grid_to_pieces(horizontal_edges, vertical_edges, opposite)

## 6. 4x4 corners-only

In [31]:
# This will be a 4x4 grid
H, W = 4, 4
horizontal_edges, vertical_edges = sample_random_grid(H, W, np.array([FLAT_ALTERNATIVE]))
horizontal_edges[:, 2] = FLAT
vertical_edges[2, :] = FLAT

# First, make sure that the overall layout can only be solved one way
assert is_unique(horizontal_edges, vertical_edges)

# Show it, to illustrate the purpose
display_grid(horizontal_edges, vertical_edges)

In [32]:
# Choose edge types
edge_types = np.array([
    CENTERED_SMALL_MALE,
    LEFT_SMALL_MALE,
    #DOUBLE_SMALL_MALE,
    CENTERED_MEDIUM_MALE,
])

# Automatically expand types to match combinations
edge_types = expand_edge_types(edge_types, opposite, flip)

# Show actual types
edge_names[edge_types]

array(['CENTERED_MEDIUM_MALE', 'CENTERED_MEDIUM_FEMALE',
       'CENTERED_SMALL_MALE', 'CENTERED_SMALL_FEMALE', 'RIGHT_SMALL_MALE',
       'RIGHT_SMALL_FEMALE', 'LEFT_SMALL_MALE', 'LEFT_SMALL_FEMALE'],
      dtype='<U22')

In [33]:
# `is_unique` will not work here, as sub-puzzles may be arranged in too many ways
# We will exploit the fact that we need to make exactly 4 unique sub-puzzles

from collections import Counter

def is_okay(horizontal_edges, vertical_edges):
    pieces = grid_to_pieces(horizontal_edges, vertical_edges, opposite)
    iterator = iterate_unique_solutions(2, 2, pieces, opposite, flip, constraints="border")
    grids = []
    for grid in iterator:
        grids.append(grid)
        if len(grids) > 4:
            return False
    if len(grids) < 4:
        return False
    
    # Take original set of pieces
    available = Counter(map(tuple, canonize_piece(pieces.reshape(-1, 4), flip)))
    
    # Take generated sub-jigsaws
    used = Counter()
    for h, v in grids:
        ps = canonize_piece(grid_to_pieces(h, v, opposite), flip)
        for p in ps.reshape(-1, 4):
            used[tuple(p)] += 1
    
    # This must match
    return available == used

In [34]:
# Generate grid
H, W = 4, 4
generator = np.random.default_rng(123456789)
def sampler():
    horizontal_edges, vertical_edges = sample_random_grid(H, W, edge_types, generator=generator)
    
    # Add a single flat edge, so that the top edge pieces become corners
    horizontal_edges[:, 2] = FLAT
    vertical_edges[2, :] = FLAT
    
    horizontal_edges[0, 1] = TWISTED_SMALL_MALE
    horizontal_edges[0, 3] = TWISTED_SMALL_FEMALE
    vertical_edges[1, 1] = TWISTED_SMALL_MALE
    
    horizontal_edges[3, 1] = LEFT_SMALL_MALE
    horizontal_edges[3, 3] = RIGHT_SMALL_MALE
    
    #vertical_edges[3, 0] = LEFT_SMALL_MALE
    #vertical_edges[3, 2] = LEFT_SMALL_FEMALE
    
    return horizontal_edges, vertical_edges
horizontal_edges, vertical_edges = search(sampler, has_unique_pieces, is_okay)

9539it [00:16, 568.80it/s]


In [35]:
# Show it
display_grid(horizontal_edges, vertical_edges)

In [36]:
# Keep it for later
pieces_4x4_only_corners = grid_to_pieces(horizontal_edges, vertical_edges, opposite)

## 7. 4x4 cycle

In [37]:
# This will be a 4x4 grid
H, W = 4, 4
horizontal_edges, vertical_edges = sample_random_grid(H, W, np.array([FLAT_ALTERNATIVE]))
vertical_edges[1, 1:-1] = FLAT
vertical_edges[2, 2:] = FLAT
vertical_edges[3, 1:-1] = FLAT
horizontal_edges[1:-1, 1] = FLAT

# Show it, to illustrate the purpose
display_grid(horizontal_edges, vertical_edges)

In [38]:
# This design has actually 3 possible layouts
display_solutions(horizontal_edges, vertical_edges)

In [39]:
# Choose edge types
edge_types = np.array([
    CENTERED_SMALL_MALE,
    LEFT_SMALL_MALE,
    #DOUBLE_SMALL_MALE,
])

# Automatically expand types to match combinations
edge_types = expand_edge_types(edge_types, opposite, flip)

# Show actual types
edge_names[edge_types]

array(['CENTERED_SMALL_MALE', 'CENTERED_SMALL_FEMALE', 'RIGHT_SMALL_MALE',
       'RIGHT_SMALL_FEMALE', 'LEFT_SMALL_MALE', 'LEFT_SMALL_FEMALE'],
      dtype='<U22')

In [40]:
# Generate grid
H, W = 4, 4
generator = np.random.default_rng(42)
def sampler():
    horizontal_edges, vertical_edges = sample_random_grid(H, W, edge_types, generator=generator)
    
    # Add walls
    vertical_edges[1, 1:-1] = FLAT
    vertical_edges[2, 2:] = FLAT
    vertical_edges[3, 1:-1] = FLAT
    horizontal_edges[1:-1, 1] = FLAT
    
    # Add some specific constraints, to force the desired pattern
    horizontal_edges[2, 2] = TWISTED_SMALL_MALE
    vertical_edges[2, 1] = TWISTED_SMALL_FEMALE
    vertical_edges[3, 3] = TWISTED_SMALL_MALE
    horizontal_edges[2, 3] = DOUBLE_SMALL_FEMALE
    horizontal_edges[3, 3] = DOUBLE_SMALL_MALE
    
    return horizontal_edges, vertical_edges
horizontal_edges, vertical_edges = search(sampler, has_unique_pieces, is_unique)

1569it [00:03, 468.48it/s]


In [41]:
# Show it
display_grid(horizontal_edges, vertical_edges)

In [42]:
# Keep it for later
pieces_4x4_cycle = grid_to_pieces(horizontal_edges, vertical_edges, opposite)

## Export as SVG

In [43]:
# Export jigsaw puzzles separately
for name, pieces in {
    "01_pieces_4x4_cross": pieces_4x4_cross,
    "02_5x5_nested": pieces_5x5_nested,
    "03_4x4_pure": pieces_4x4_pure,
    "04_4x4_edge": pieces_4x4_edge,
    "05_4x4_6_corners": pieces_4x4_6_corners,
    "06_4x4_only_corners": pieces_4x4_only_corners,
    "07_4x4_cycle": pieces_4x4_cycle,
}.items():
    horizontal_edges, vertical_edges = pieces_to_grid(pieces, opposite)
    display_grid(horizontal_edges, vertical_edges)
    with open(f"image/02_{name}.svg", "w") as file:
        file.write(grid_to_svg(horizontal_edges, vertical_edges, margin=1 / 25))

In [44]:
# Or pack jigsaw puzzles as a single grid
pieces = np.zeros((8, 8, 4), dtype=np.uint8)
pieces[:4, :4] = pieces_4x4_cycle
pieces[4:, :4] = pieces_4x4_6_corners
pieces[:4, 4:] = pieces_4x4_edge
pieces[4:, 4:] = pieces_4x4_pure
horizontal_edges, vertical_edges = pieces_to_grid(pieces, opposite)
display_grid(horizontal_edges, vertical_edges)
with open("image/02.svg", "w") as file:
    file.write(grid_to_svg(horizontal_edges, vertical_edges, margin=1 / 25))